In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install fastai==0.7.0
!pip install torchtext==0.2.3
import os

os.makedirs("data\mercari",exist_ok=True)
os.makedirs(".kaggle",exist_ok=True)
!pip install kaggle
#!mkdir .kaggle

import json 
token = {"username":"hduongck","key":"983e2ab1fbb29cf2734bcbf8811d42fb"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
 
!chmod 600 /content/.kaggle/kaggle.json

!kaggle config set -n path -v{/content/data}

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle competitions download -c mercari-price-suggestion-challenge -p /content/data/mercari

- path is now set to: {/content/data}
404 - Not Found


In [0]:
!pip install pyunpack
!pip install patool

  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
  Stored in directory: /root/.cache/pip/wheels/41/22/19/af15ef6264c58b625a82641ed7483ad05e258fbd8925505227
Successfully built pyunpack easyprocess
    100% |████████████████████████████████| 81kB 3.2MB/s 


In [0]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from IPython.display import display

In [0]:
PATH = 'data/mercari/'
!ls {PATH}

sample_submission.csv.7z	test_stg2.tsv.zip  train.tsv.7z
sample_submission_stg2.csv.zip	test.tsv.7z


In [0]:
from pyunpack import Archive
import os
#listfiles = os.listdir(PATH)
listfiles = ['train.tsv.7z','test.tsv.7z']
for file in listfiles:
    Archive(PATH+file).extractall(PATH)

# Data processing

In [0]:
table_names = ['train','test']

In [0]:
tables = [pd.read_table(f'{PATH}{fname}.tsv',low_memory=True,sep='\t') for fname in table_names]

In [0]:
for t in tables : display(t.head())

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [0]:
train, test = tables

In [0]:
train.dtypes

train_id               int64
name                  object
item_condition_id      int64
category_name         object
brand_name            object
price                float64
shipping               int64
item_description      object
dtype: object

In [0]:
train.category_name.value_counts()

Women/Athletic Apparel/Pants, Tights, Leggings                 60177
Women/Tops & Blouses/T-Shirts                                  46380
Beauty/Makeup/Face                                             34335
Beauty/Makeup/Lips                                             29910
Electronics/Video Games & Consoles/Games                       26557
Beauty/Makeup/Eyes                                             25215
Electronics/Cell Phones & Accessories/Cases, Covers & Skins    24676
Women/Underwear/Bras                                           21274
Women/Tops & Blouses/Tank, Cami                                20284
Women/Tops & Blouses/Blouse                                    20284
Women/Dresses/Above Knee, Mini                                 20082
Women/Jewelry/Necklaces                                        19758
Women/Athletic Apparel/Shorts                                  19528
Beauty/Makeup/Makeup Palettes                                  19103
Women/Shoes/Boots                 

In [0]:
train.brand_name.value_counts().index

CategoricalIndex(['PINK', 'Nike', 'Victoria's Secret', 'LuLaRoe', 'Apple',
                  'FOREVER 21', 'Nintendo', 'Lululemon', 'Michael Kors',
                  'American Eagle',
                  ...
                  'KMS California', 'KR3W', 'KRW', 'Kae Argatherapie', 'Kaii',
                  'Kailijumei Lipsticks', 'Karen Neuburger', 'Karma',
                  'Kate Aspen', '!iT Jeans'],
                 categories=['!iT Jeans', '% Pure', '10.Deep', '191 Unlimited', '1byone', '21men', '24/7 Comfort Apparel', '2K Games', ...], ordered=True, dtype='category', length=4809)

In [0]:
train[train.brand_name=='Nike']

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3
8,8,Girls Nike Pro shorts,3,Nike,19.0,0,Girls Size small Plus green. Three shorts total.,Sports & Outdoors,Apparel,Girls
89,89,Nike boy sneakers,3,Nike,28.0,0,Size 6Y. High tops. Super light. In good condi...,Kids,Boys (4+),Shoes
116,116,Lebron soldier 9s size 7,2,Nike,51.0,0,Boys lebron soldier 9s great condition,Men,Shoes,Athletic
151,151,Nike men's dri-fit sleeveless shirt tee,3,Nike,14.0,0,This is a men's Nike dri-fit shirt which is bl...,Men,Tops,T-shirts
172,172,Toddler Nike tennis shoes,3,Nike,10.0,0,No description yet,Kids,Boys 0-24 Mos,Shoes
198,198,NIKE Air Max 95 Essentials,2,Nike,54.0,0,FLASH SALE !! TODAY (Sunday) ONLY !! 20% OFF L...,Men,Shoes,Athletic
298,298,Jordan's kids size 6,4,Nike,14.0,0,No description yet,Kids,Boys 0-24 Mos,Shoes
312,312,Men's Nike force shoes,4,Nike,18.0,0,Some scuffs and creases,Men,Shoes,Athletic
327,327,Nike dri fit leggings size small,4,Nike,25.0,0,Good condition has small pocket on back and zi...,Women,Athletic Apparel,"Pants, Tights, Leggings"
339,339,Nike Pros,3,Nike,13.0,0,Dri-Fit The bottom of the N wore off a little,Women,Athletic Apparel,Shorts


# Split category_name column into 3 sub-col

In [0]:
def cat_split(row):
    try :
        text=row
        txt1,txt2,txt3 = text.split('/')
        return txt1,txt2,txt3
    except:
        return np.nan,np.nan,np.nan

In [0]:
train['cat_1'],train['cat_2'],train['cat_3'] = zip(*train.category_name.apply(lambda val : cat_split(val)))

In [0]:
train = train.drop(['category_name'],axis=1)

In [0]:
train.head(10)

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3
0,0,MLB Cincinnati Reds T Shirt Size XL,3,NaN,10.0,1,No description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces
5,5,Bundled items requested for Ruie,3,NaN,59.0,0,"Banana republic bottoms, Candies skirt with ma...",Women,Other,Other
6,6,Acacia pacific tides santorini top,3,Acacia Swimwear,64.0,0,Size small but straps slightly shortened to fi...,Women,Swimwear,Two-Piece
7,7,Girls cheer and tumbling bundle of 7,3,Soffe,6.0,1,You get three pairs of Sophie cheer shorts siz...,Sports & Outdoors,Apparel,Girls
8,8,Girls Nike Pro shorts,3,Nike,19.0,0,Girls Size small Plus green. Three shorts total.,Sports & Outdoors,Apparel,Girls
9,9,Porcelain clown doll checker pants VTG,3,NaN,8.0,0,I realized his pants are on backwards after th...,Vintage & Collectibles,Collectibles,Doll


In [0]:
train.cat_1.value_counts()

Women                     664385
Beauty                    207828
Kids                      171689
Electronics               119631
Men                        93363
Home                       67871
Vintage & Collectibles     46530
Other                      45351
Handmade                   30840
Sports & Outdoors          24331
Name: cat_1, dtype: int64

In [0]:
train.cat_3.value_counts()

Pants, Tights, Leggings       60177
Other                         50224
Face                          50171
T-Shirts                      46380
Shoes                         32168
Games                         30906
Lips                          30871
Athletic                      27059
Eyes                          26038
Cases, Covers & Skins         24676
Shorts                        21815
Bras                          21274
Blouse                        20288
Tank, Cami                    20284
Boots                         20270
Above Knee, Mini              20082
Necklaces                     19758
Makeup Palettes               19103
Women                         18975
Shirts & Tops                 16876
Sandals                       15368
Shoulder Bag                  15328
Fashion Sneakers              15203
T-shirts                      15108
Wallets                       14884
Knee-Length                   14770
Tops & T-Shirts               14508
Bracelets                   

In [0]:
test['cat_1'],test['cat_2'],test['cat_3'] = zip(*test.category_name.apply(lambda val : cat_split(val)))

In [0]:
test = test.drop(['category_name'],axis=1)

In [0]:
train_cats(train)

In [0]:
apply_cats(test,train)

In [0]:
df_train,y,nas = proc_df(train,'price',skip_flds=['train_id'])

In [0]:
df_train.head(10)

,item_condition_id,brand_name,shipping,cat_1,cat_2,cat_3
0,3,0,1,6,103,759
1,3,3558,0,2,31,213
2,1,4181,1,10,104,95
3,1,0,1,4,56,402
4,1,0,0,10,59,533
5,3,0,0,10,73,555
6,3,80,0,10,99,820
7,3,3958,1,8,3,351
8,3,3058,0,8,3,351
9,3,0,0,9,30,258


In [0]:
df_train = df_train.drop(['item_description','name'],axis=1)

In [0]:
y = np.log1p(np.clip(y,0,None))

In [0]:
def split_vals(a,n): return a[:n].copy(),a[n:].copy()

In [0]:
n_valid = len(test)
n_trn = len(df_train)-n_valid
X_train,X_valid = split_vals(df_train,n_trn)
y_train,y_valid = split_vals(y,n_trn)

In [0]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train),y_train),
           rmse(m.predict(X_valid),y_valid),
           m.score(X_train,y_train),
           m.score(X_valid,y_valid)]
    if hasattr(m,'oob_score_'): res.append(m.oob_score_)
    print(res)

# Random Forest


In [0]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train,y_train)
print_score(m)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.1333174743013567, 0.14318953592819794, 0.4912951115915889, 0.41504329735315737]


In [0]:
m = RandomForestRegressor(n_jobs=-1,n_estimators=40)
m.fit(X_train,y_train)
print_score(m)

[0.21111012935993045, 0.5803409147188767, 0.9205646680769463, 0.40030399041651815]
